## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-19-06-50-09 +0000,nypost,California driver named Isis fights with DMV t...,https://nypost.com/2025/09/19/us-news/californ...
1,2025-09-19-06-42-35 +0000,nypost,Byron Donalds fires back at Rashida Tlaib afte...,https://nypost.com/2025/09/19/us-news/byron-do...
2,2025-09-19-06-42-31 +0000,bbc,Highest government borrowing in August for fiv...,https://www.bbc.com/news/articles/cly9m4lqznro...
3,2025-09-19-06-32-23 +0000,nyt,How Xi Is Using a TikTok Tradeoff to Court Trump,https://www.nytimes.com/2025/09/19/world/asia/...
4,2025-09-19-06-29-00 +0000,wsj,U.K. Retail Sales Climbed Again in August,https://www.wsj.com/economy/u-k-retail-sales-c...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2365/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,70
82,kimmel,28
231,jimmy,23
496,new,18
285,kirk,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
60,2025-09-19-00-40-36 +0000,nypost,Jimmy Kimmel raged against Trump and his ‘MAGA...,https://nypost.com/2025/09/18/us-news/jimmy-ki...,177
234,2025-09-18-15-10-04 +0000,nypost,Trump claims Jimmy Kimmel was taken off air fo...,https://nypost.com/2025/09/18/us-news/trump-cl...,176
106,2025-09-18-21-50-35 +0000,nyt,What Jimmy Kimmel’s Suspension Says About the ...,https://www.nytimes.com/2025/09/18/business/me...,159
150,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,155
159,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...,149


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
60,177,2025-09-19-00-40-36 +0000,nypost,Jimmy Kimmel raged against Trump and his ‘MAGA...,https://nypost.com/2025/09/18/us-news/jimmy-ki...
150,85,2025-09-18-20-13-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...
159,67,2025-09-18-19-53-00 +0000,wsj,The Trump administration is putting the weight...,https://www.wsj.com/politics/policy/jimmy-kimm...
72,57,2025-09-19-00-00-33 +0000,nypost,Billionaire Mets owner Steve Cohen emerges as ...,https://nypost.com/2025/09/18/us-news/billiona...
328,49,2025-09-18-08-08-57 +0000,wsj,"Stock Market News, Sept. 18, 2025: Dow Rises A...",https://www.wsj.com/livecoverage/stock-market-...
313,46,2025-09-18-09-30-00 +0000,wsj,Capital Account: Whatever probability you put ...,https://www.wsj.com/economy/central-banking/th...
134,45,2025-09-18-20-48-39 +0000,nypost,UK cops show up at Trump-supporting cancer pat...,https://nypost.com/2025/09/18/world-news/uk-co...
309,43,2025-09-18-09-30-14 +0000,wapo,"For the first time, Bernie Sanders calls Israe...",https://www.washingtonpost.com/politics/2025/0...
170,42,2025-09-18-18-59-08 +0000,bbc,Two shot dead in attack near West Bank-Jordan ...,https://www.bbc.com/news/articles/cr5qv0qmgmjo...
233,33,2025-09-18-15-15-02 +0000,nypost,Trump admits Putin ‘let me down’ on Ukraine pe...,https://nypost.com/2025/09/18/us-news/trump-ad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
